In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

2022-11-10 16:12:55.579337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 16:12:55.699479: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 16:12:56.183287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-10 16:12:56.183336: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
import tensorflow_datasets as tfds
ds, ds_info = tfds.load('organoids', split='train',batch_size=64, with_info=True)

2022-11-10 16:13:03.245013: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
for batch in ds.take(1):
    print(batch['expression'])

tf.Tensor(
[[0.         3.8617337  4.571121   ... 0.         4.8368387  7.9113965 ]
 [0.         3.0613792  5.5842743  ... 0.         5.43245    7.023813  ]
 [0.         2.2419517  3.158241   ... 0.         2.271955   4.4754205 ]
 ...
 [0.17605919 4.2487483  2.9109216  ... 5.0950274  4.776161   5.8817267 ]
 [0.29639632 2.8709605  2.5053656  ... 0.         2.811519   5.378569  ]
 [0.24135396 4.15895    3.709844   ... 0.         3.0586607  6.2270145 ]], shape=(64, 41), dtype=float32)


In [4]:
ds_info.features['cell_type'].names

['Enterocyte', 'Enteroendocrine', 'Goblet', 'Paneth', 'Stem', 'Tuft']

In [201]:
from functools import partial
from typing import Sequence, Callable, Any

from absl import app
from absl import flags
from flax import linen as nn
from flax.training import train_state
import jax.numpy as jnp
import jax
from jax import random

ModuleDef = Any

class Encoder(nn.Module):
    latents: int
    hidden_dims: Sequence[int]
    norm:ModuleDef
    act: Callable
    @nn.compact
    def __call__(self, x):
        for i, hdim in enumerate(self.hidden_dims):
            x = nn.Dense(hdim, name=f'fc_enc_{i}_{hdim}')(x)
            x = self.act()(x)
            x = self.norm()(x)

        mean_x = nn.Dense(self.latents, name='latent_mean')(x)
        logvar_x = nn.Dense(self.latents, name='latent_logvar')(x)
        return mean_x, logvar_x


class Decoder(nn.Module):
    hidden_dims: Sequence[int]
    input_features: int
    norm:ModuleDef
    act: Callable

    @nn.compact
    def __call__(self, z):
        for i, hdim in enumerate(reversed(self.hidden_dims)):
            z = nn.Dense(hdim, name=f'fc_dec_{i}_{hdim}')(z)
            z = self.act()(z)
            z = self.norm()(z)

        z = nn.Dense(self.input_features, name='fc_dec_last')(z)
        return z


class VAE(nn.Module):
    latents: int
    hidden_dims: Sequence[int]
    input_features: int
    act: Callable = nn.PReLU
    encoder_cls = Encoder
    decoder_cls = Decoder
    dtype: Any = jnp.float32

    @nn.compact
    def __call__(self, x, z_rng, train = True):
        norm = partial(nn.BatchNorm,
                   use_running_average=not train,
                   momentum=0.9,
                   epsilon=1e-5,
                   dtype=self.dtype)

        mean, logvar = self.encoder_cls(self.latents,
                                    self.hidden_dims,
                                    act=self.act,
                                    norm=norm)(x)

        z = reparameterize(z_rng, mean, logvar)

        recon_x = self.decoder_cls(self.hidden_dims,
                               self.input_features,
                               act=self.act,
                               norm=norm)(z)

        return recon_x, mean, logvar

    def generate(self, z):
        return self.decoder(z)

class TrainState(train_state.TrainState):
  batch_stats: Any

def reparameterize(rng, mean, logvar):
    std = jnp.exp(0.5 * logvar)
    eps = random.normal(rng, logvar.shape)
    return mean + eps * std

In [46]:
from flax import jax_utils

def prepare_tf_data(xs):
  """Convert a input batch from tf Tensors to numpy arrays."""
  local_device_count = jax.local_device_count()
  def _prepare(x):
    # Use _numpy() for zero-copy conversion between TF and NumPy.
    x = x._numpy()  # pylint: disable=protected-access

    # reshape (host_batch_size, height, width, 3) to
    # (local_devices, device_batch_size, height, width, 3)
    return x.reshape((local_device_count, -1) + x.shape[1:])

  return jax.tree_util.tree_map(_prepare, xs)


In [106]:
test_encoder = Encoder(latents=2, hidden_dims=[32,16,8],
                     norm = partial(nn.BatchNorm,use_running_average=True),
                     act = nn.PReLU)
variables = test_encoder.init(random.PRNGKey(0), jnp.ones((5,5)))
y = test_encoder.apply(variables, jnp.ones((5,5)))

(32, 16, 8) 0 32
(5, 5)
(32, 16, 8) 1 16
(5, 32)
(32, 16, 8) 2 8
(5, 16)
(32, 16, 8) 0 32
(5, 5)
(32, 16, 8) 1 16
(5, 32)
(32, 16, 8) 2 8
(5, 16)


In [102]:
y

(DeviceArray([[-0.56667733,  0.16567218],
              [-0.56667733,  0.16567218],
              [-0.56667733,  0.16567218],
              [-0.56667733,  0.16567218],
              [-0.56667733,  0.16567218]], dtype=float32),
 DeviceArray([[0.3108176 , 0.15926342],
              [0.3108176 , 0.15926342],
              [0.3108176 , 0.15926342],
              [0.3108176 , 0.15926342],
              [0.3108176 , 0.15926342]], dtype=float32))

In [114]:
test_decoder = Decoder(hidden_dims=[32,16,8],
                       input_features=41,
                       norm = partial(nn.BatchNorm,use_running_average=True),
                       act = nn.PReLU)
variables = test_decoder.init(random.PRNGKey(0), jnp.ones((5,2)))
y = test_decoder.apply(variables, jnp.ones((5,2)))

(32, 16, 8) 0 8
(5, 2)
(32, 16, 8) 1 16
(5, 8)
(32, 16, 8) 2 32
(5, 16)
(32, 16, 8) 0 8
(5, 2)
(32, 16, 8) 1 16
(5, 8)
(32, 16, 8) 2 32
(5, 16)


In [116]:
y.shape

(5, 41)

In [131]:
test_vae = model()
variables = test_vae.init(random.PRNGKey(0), jnp.ones((5,41)),random.PRNGKey(1))

[32 16  8] 0 32
(5, 41)
[32 16  8] 1 16
(5, 32)
[32 16  8] 2 8
(5, 16)
[32 16  8] 0 8
(5, 2)
[32 16  8] 1 16
(5, 8)
[32 16  8] 2 32
(5, 16)


In [135]:
y = test_vae.apply(variables, jnp.ones((5,41)),random.PRNGKey(2),train=False)

[32 16  8] 0 32
(5, 41)
[32 16  8] 1 16
(5, 32)
[32 16  8] 2 8
(5, 16)
[32 16  8] 0 8
(5, 2)
[32 16  8] 1 16
(5, 8)
[32 16  8] 2 32
(5, 16)


In [138]:
[i.shape for i in y]

[(5, 41), (5, 2), (5, 2)]

In [140]:
@jax.vmap
def kl_divergence(mean, logvar):
    return -0.5 * jnp.sum(1 + logvar - jnp.square(mean) - jnp.exp(logvar))

@jax.vmap
def mse(input, reconstructed):
    return jnp.sum((input - reconstructed) ** 2)


def compute_metrics(recon_x, x, mean, logvar):
    mse_loss = mse(recon_x, x).mean()
    kld_loss = kl_divergence(mean, logvar).mean()
    return {
        'mse': mse_loss,
        'kld': kld_loss,
        'loss': mse_loss + kld_loss * 0.0025
    }

In [241]:
@jax.jit
def train_step(state, batch, z_rng):
    def loss_fn(params):
        print("bshape",batch.shape)
        (recon_x, mean, logvar) , _ = model().apply({'params': params, 'batch_stats': state.batch_stats},
                                              batch, z_rng, mutable=['batch_stats'])
        print(recon_x.shape, mean.shape, logvar.shape)
        mse_loss = mse(recon_x, batch).mean()
        kld_loss = kl_divergence(mean, logvar).mean()
        loss = mse_loss + kld_loss * 0.0025
        return loss

    grads = jax.grad(loss_fn)(state.params)
    return state.apply_gradients(grads=grads)


@jax.jit
def eval(params, x, z, z_rng):
    def eval_model(vae):
        x_recon, mean, logvar = vae(x, z_rng, train = False)

        metrics = compute_metrics(x_recon, x, mean, logvar)
        return metrics, z

    return nn.apply(eval_model, model())({'params': params, 'batch_stats': state.batch_stats})

In [235]:
rng = random.PRNGKey(0)
rng, key = random.split(rng)

train_ds = tfds.load('organoids', split='train')
train_ds = train_ds.batch(10**7)
train_ds = train_ds.cache()
train_ds = train_ds.repeat()
train_ds = iter(tfds.as_numpy(train_ds))

test_ds = tfds.load('organoids', split='validation')
test_ds = test_ds.batch(10**6)

In [149]:
next(train_ds)['expression'].shape

(937980, 41)

In [182]:
import numpy as np
def model():
    return VAE(latents=2,
               hidden_dims=np.array([32,16,8]),
               input_features=41)

In [246]:
def initialized(key, model):
  input_shape = (1, 41)
  @jax.jit
  def init(*args):
    return model.init(*args)
  variables = init({'params': key}, jnp.ones(input_shape, model.dtype), key)
  return variables['params'], variables['batch_stats']

import optax

init_data = jnp.ones((937980, 41), jnp.float32)

params, batch_stats = initialized(rng,model())
state = TrainState.create(
      apply_fn=model().apply,
      params=params,
      tx=optax.adam(0.01),
      batch_stats=batch_stats,
)

rng, z_key, eval_rng = random.split(rng, 3)
z = random.normal(z_key, (64, 2))
steps_per_epoch = 1

for epoch in range(100):
    for _ in range(steps_per_epoch):
        batch = next(train_ds)['expression']
        rng, key = random.split(rng)
        state = train_step(state, batch, key)

        metrics, _ = eval(state.params, next(iter(test_ds))['expression']._numpy(), z, eval_rng)

        print('eval epoch: {}, loss: {:.4f}, MSE: {:.4f}, KLD: {:.4f}'.format(
            epoch + 1, metrics['loss'], metrics['mse'], metrics['kld']
        ))

bshape (937980, 41)
(937980, 41) (937980, 2) (937980, 2)
eval epoch: 1, loss: 246.7663, MSE: 246.7651, KLD: 0.4669
eval epoch: 2, loss: 244.3150, MSE: 244.3130, KLD: 0.7980
eval epoch: 3, loss: 241.8796, MSE: 241.8768, KLD: 1.1182
eval epoch: 4, loss: 239.4857, MSE: 239.4822, KLD: 1.3969
eval epoch: 5, loss: 236.7507, MSE: 236.7468, KLD: 1.5665
eval epoch: 6, loss: 233.6154, MSE: 233.6112, KLD: 1.6602
eval epoch: 7, loss: 230.2632, MSE: 230.2588, KLD: 1.7432
eval epoch: 8, loss: 226.7554, MSE: 226.7508, KLD: 1.8462
eval epoch: 9, loss: 223.0200, MSE: 223.0150, KLD: 1.9846
eval epoch: 10, loss: 218.8901, MSE: 218.8847, KLD: 2.1591
eval epoch: 11, loss: 214.2507, MSE: 214.2448, KLD: 2.3614
eval epoch: 12, loss: 209.0384, MSE: 209.0319, KLD: 2.5764
eval epoch: 13, loss: 203.1673, MSE: 203.1603, KLD: 2.7790
eval epoch: 14, loss: 196.4226, MSE: 196.4152, KLD: 2.9715
eval epoch: 15, loss: 188.6123, MSE: 188.6043, KLD: 3.1871
eval epoch: 16, loss: 179.8338, MSE: 179.8250, KLD: 3.4855
eval epo

In [233]:
next(iter(test_ds))['expression']